<a href="https://colab.research.google.com/github/Omm-Sharma/wHackiest2024_Brainybytes_Mental_Health_chatbot/blob/main/wHACKIEST2024_Mental_Health_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!pip install langchain_groq

In [29]:
!pip install sentence-transformers

  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [24]:
!pip install chromadb

  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


In [25]:
!pip install langchain_core langchain_community

In [26]:
!pip install gradio

In [27]:
!pip install pypdf

In [31]:
# Install required dependencies
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
import os
import gradio as gr

# Set API key directly (replace with your key)
GROQ_API_KEY = "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"

def initialize_llm():
    """Initialize the ChatGroq LLM with shorter response constraints."""
    return ChatGroq(
        temperature=0,
        groq_api_key=GROQ_API_KEY,
        model_name="llama-3.3-70b-versatile",
        max_tokens=250  # Limit response length
    )

def create_vector_db(data_dir="/content/data", db_dir="/content/chroma_db"):
    """Load PDFs, split text, create embeddings, and save the vector database."""
    if not os.path.exists(data_dir):
        raise FileNotFoundError(f"Data directory '{data_dir}' not found.")

    loader = DirectoryLoader(data_dir, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma.from_documents(texts, embeddings, persist_directory=db_dir)
    vector_db.persist()

    print("ChromaDB created and data saved.")
    return vector_db

def setup_qa_chain(vector_db, llm):
    """Set up the retrieval QA chain with a concise prompt template."""
    retriever = vector_db.as_retriever()
    prompt_template = """
    You are a compassionate mental health chatbot. Respond to questions and thoughtfully to the following question:
    {context}
    User: {question}
    Chatbot (in 4-5 sentences):"""

    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

    return RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={"prompt": prompt}
    )

# Initialize the chatbot
print("Initializing Chatbot...")
llm = initialize_llm()
db_path = "/content/chroma_db"

# Ensure the data directory exists
data_dir = "/content/data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Created data directory at '{data_dir}'. Please upload PDFs to this directory.")

# Create vector database if it doesn't exist
if not os.path.exists(db_path):
    vector_db = create_vector_db(data_dir, db_path)
else:
    embeddings = HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)

qa_chain = setup_qa_chain(vector_db, llm)

def chatbot_response(user_input, history=[]):
    """Generate a chatbot response and update the conversation history."""
    if not user_input.strip():
        return "Please enter a valid query.", history

    response = qa_chain.run(user_input)
    history.append((user_input, response))
    return "", history

# Gradio interface setup
with gr.Blocks(title="Mann - Mental Support Bot") as app:
    gr.Markdown(
        """
        # 🧠 **Mann - Your Mental Health Support Bot**
        ---
        **Ask any mental health-related questions, and Mann will respond thoughtfully and concisely.**
        """
    )
    chatbot = gr.Chatbot(height=400)
    user_input = gr.Textbox(
        placeholder="Type your question here...",
        label="Your Query"
    )
    send_button = gr.Button("Send 💬")
    clear_button = gr.Button("Clear 🗑️")

    def process_input(message, history):
        response = qa_chain.run(message)
        history.append((message, response))
        return "", history

    send_button.click(process_input, inputs=[user_input, chatbot], outputs=[user_input, chatbot])
    clear_button.click(lambda: ([], []), outputs=[chatbot, user_input])

app.launch(share=True)

Initializing Chatbot...


<ipython-input-31-0a47a12b7ceb>:76: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(persist_directory=db_path, embedding_function=embeddings)
/usr/local/lib/python3.10/dist-packages/gradio/components/chatbot.py:242: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://20f92f2379c0710663.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
